# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
S. Kurinchi-Vendhan  ->  S. Kurinchi-Vendhan  |  ['S. Kurinchi-Vendhan']
E. Rohr  ->  E. Rohr  |  ['E. Rohr']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
M. Sharma  ->  Y. M. Sharma  |  ['M. Sharma']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 58 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.05459


extracting tarball to tmp_2506.05459...

 done.


Found 103 bibliographic references in tmp_2506.05459/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.05474


...

extracting tarball to tmp_2506.06149...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure .png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1\columnwidth]{{fmol_thorhr_thor}.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure .png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=1\columnwidth]{{xoh_vs_rgc}.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-pa

Found 114 bibliographic references in tmp_2506.06149/oh_highres_paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.06164
extracting tarball to tmp_2506.06164...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05459-b31b1b.svg)](https://arxiv.org/abs/2506.05459) | **RUBIES: A Spectroscopic Census of Little Red Dots; All V-Shaped Point Sources Have Broad Lines**  |
|| R. E. Hviding, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *Main text: 17 pages, 10 figures. Appendix: 7 pages, 3 figures. Submitted to A&A*|
|**Abstract**|            The physical nature of Little Red Dots (LRDs) - a population of compact, red galaxies revealed by JWST - remains unclear. Photometric samples are constructed from varying selection criteria with limited spectroscopic follow-up available to test intrinsic spectral shapes and prevalence of broad emission lines. We use the RUBIES survey, a large spectroscopic program with wide color-morphology coverage and homogeneous data quality, to systematically analyze the emission-line kinematics, spectral shapes, and morphologies of $\sim$1500 galaxies at $z > 3.1$. We identify broad Balmer lines via a novel fitting approach that simultaneously models NIRSpec/PRISM and G395M spectra, yielding 80 broad-line sources with 28 (35%) at $z > 6$. A large subpopulation naturally emerges from the broad Balmer line sources, with 36 exhibiting `v-shaped' UV-to-optical continua and a dominant point source component in the rest-optical; we define these as spectroscopic LRDs, constituting the largest such sample to date. Strikingly, the spectroscopic LRD population is largely recovered when either a broad line or rest-optical point source is required in combination with a v-shaped continuum, suggesting an inherent link between these three defining characteristics. We compare the spectroscopic LRD sample to published photometric searches. Although these selections have high accuracy, down to $\rm F444W<26.5$, only 50-62% of the RUBIES LRDs were previously identified. The remainder were missed due to a mixture of faint rest-UV photometry, comparatively blue rest-optical colors, or highly uncertain photometric redshifts. Our findings highlight that well-selected spectroscopic campaigns are essential for robust LRD identification, while photometric criteria require refinement to capture the full population.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05474-b31b1b.svg)](https://arxiv.org/abs/2506.05474) | **Jellyfish galaxies with the IllustrisTNG simulations -- Supermassive black hole activity in dense environments with ram-pressure stripped satellites**  |
|| <mark>S. Kurinchi-Vendhan</mark>, et al. -- incl., <mark>E. Rohr</mark>, <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *21 pages, 9 figures + 3 appendices with 3 figures. Submitted to MNRAS*|
|**Abstract**|            Jellyfish galaxies are extreme examples of how galaxies can transform due to dense environmental effects. These satellite galaxies suffer from ram-pressure stripping, leading to the formation of their distinctive gaseous tails. Some recent observational studies find that jellyfish galaxies are more likely to host active galactic nuclei (AGN) compared to central galaxies of the same mass, suggesting a link between ram pressure and supermassive black hole (SMBH) accretion. We use the IllustrisTNG cosmological-magnetohydrodynamical simulations, namely TNG50 and TNG100, to explore the presence of AGN in jellyfish galaxies with $M_{\rm{stellar}}\simeq10^{9.5-10.8}\,\rm{M}_\odot$ at redshift $z=0$ from the Zooniverse "Cosmological Jellyfish" citizen-science project. Compared to central galaxies, jellyfish are more likely to host an AGN ($L_{\rm AGN}\geq10^{44}\,\mathrm{erg\,s^{-1}}$) particularly at high stellar masses ($M_{\rm stellar}\gtrsim10^{10}\,\mathrm{M_\odot}$). Jellyfish are also more likely to host an AGN than satellites of the same mass, largely because many satellite galaxies are gas-poor and therefore have lower SMBH accretion rates. Compared to non-jellyfish satellites with similar gas content, jellyfish typically undergo stronger ram pressure and have higher central gas densities along with lower central gas sound speeds, although these effects are smaller at lower stellar masses ($M_{\rm stellar}\lesssim10^{10}\,\mathrm{M_\odot}$). Together with case studies of individual galaxies, our population analysis indicates that ram pressure can play a key role in fuelling AGN activity in a large fraction of jellyfish, where gas compression can lead to intense episodes of AGN feedback and star formation. Thus, it is essential to consider both environmental and secular processes for a more complete picture of satellite galaxy evolution.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06014-b31b1b.svg)](https://arxiv.org/abs/2506.06014) | **Dynamical mass distribution and velocity structure of the Galactic centre**  |
|| A. Feldmeier-Krause, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *15 pages (+ 5 pages Appendix), 10 (+ 5) figures, accepted A&A*|
|**Abstract**|            The inner ~200 pc region of the Milky Way contains a nuclear stellar disc and a nuclear star cluster that are embedded in the larger Galactic bar. These stellar systems overlap spatially, which makes it challenging to separate stars that belong to the nuclear stellar systems, to deduce their internal dynamics, and to derive the central Galactic potential. Discrete stellar kinematics probe the mass distribution of a stellar system, and chemical tracers such as stellar metallicity can further separate multiple stellar populations that can have distinct kinematic properties. We took advantage of the information provided by discrete stellar kinematics and the metallicity in the Galactic centre using discrete chemo-dynamical modelling. We fitted axisymmetric Jeans models to discrete data of 4,600 stars. We fitted the stars as either one population plus a background component or as two populations plus a background that represents the bar. We tested the robustness of the inferred gravitational potential against a varying mass of the supermassive black hole, including dark matter, or a radially varying mass-to-light ratio. We obtained robust results on the fit with a single population and a background component. We obtained a supermassive black hole mass of (4.35$\pm 0.24) \times 10^6$ M$_\odot$, and we find that a dark matter component and radial variation in the mass-to-light ratio are negligible. We derived the enclosed mass profile of the inner ~60 pc and found a lower mass than reported in the literature in the region of ~5-30 pc. In our two-population fit, we found a high-[M/H] population that contributes more than 90% to the total stellar density. The properties of the high-[M/H] population are consistent with in situ formation after gas inflow from the Galactic disc via the bar. The distinct kinematic properties of the low-[M/H] population indicate a different origin. [abridged]         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06149-b31b1b.svg)](https://arxiv.org/abs/2506.06149) | **Faint absorption of the ground state hyperfine-splitting transitions of hydroxyl at 18 cm in the Galactic Disk**  |
|| M. R. Rugel, et al. -- incl., <mark>H. Beuther</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *13 pages, 7 figures (appendices excluded). Accepted for publication in A&A*|
|**Abstract**|            The interstellar hydride hydroxyl (OH) is a potential tracer of CO-dark molecular gas. We present new absorption line observations of OH at 18-cm wavelength towards four continuum sources. We compare these to the [CII] line at 1.9 THz obtained with SOFIA, observations of the neutral atomic hydrogen 21 cm line with the VLA, and CO lines obtained with APEX. We trace OH over a large range of molecular hydrogen column densities, and derive OH abundances with respect to molecular and total hydrogen column densities. Increased sensitivity and spectral resolution allowed us to detect weak and narrow features. We identify only one OH absorption component out of 23 without CO counterpart, yet several with intermediate molecular gas fractions. A potential association of [CII] 158 mu m emission with an OH absorption component is seen toward one sightline. Our results confirm that OH absorption traces molecular gas across diffuse and dense environments of the interstellar medium. At the sensitivity limits of the present observations our detection of only one CO-dark molecular gas feature appears in agreement with previous studies. We conclude that if OH absorption was to be used as a CO-dark molecular gas tracer, deeper observations or stronger background targets are necessary to unveil its full potential as a CO-dark molecular gas tracer, and yet it will never be an exclusive tracer of CO-dark molecular gas. For OH hyperfine-splitting transitions in the vicinity of photodissociation regions in W43-South, we detect a spectral and spatial offset between the peak of the inversion of the OH 1612 MHz line and the absorption of the OH 1720 MHz line on the one hand, and the absorption of the OH main lines on the other hand, which provides additional constraints on the interpretation of the OH 18 cm line signatures typical of HII regions.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05631-b31b1b.svg)](https://arxiv.org/abs/2506.05631) | **The TESS Ten Thousand Catalog: 10,001 uniformly-vetted and -validated Eclipsing Binary Stars detected in Full-Frame Image data by machine learning and analyzed by citizen scientists**  |
|| V. B. Kostov, et al. -- incl., <mark>M. Sharma</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *40 pages, 39 figures, 4 tables*|
|**Abstract**|            The Transiting Exoplanet Survey Satellite (TESS) has surveyed nearly the entire sky in Full-Frame Image mode with a time resolution of 200 seconds to 30 minutes and a temporal baseline of at least 27 days. In addition to the primary goal of discovering new exoplanets, TESS is exceptionally capable at detecting variable stars, and in particular short-period eclipsing binaries which are relatively common, making up a few percent of all stars, and represent powerful astrophysical laboratories for deep investigations of stellar formation and evolution. We combed Sectors 1-82 of TESS Full-Frame Image data searching for eclipsing binary stars using a neural network that identified ~1.2 million stars with eclipse-like features. Of these, we have performed an in-depth analysis on ~60,000 targets using automated methods and manual inspection by citizen scientists. Here we present a catalog of 10001 uniformly-vetted and -validated eclipsing binary stars that passed all our ephemeris and photocenter tests, as well as complementary visual inspection. Of these, 7936 are new eclipsing binaries while the remaining 2065 are known systems for which we update the published ephemerides. We outline the detection and analysis of the targets, discuss the properties of the sample, and highlight potentially interesting systems. Finally, we also provide a list of ~900,000 unvetted and unvalidated targets for which the neural network found eclipse-like features with a score higher than 0.9, and for which there are no known eclipsing binaries within a sky-projected separation of a TESS pixel (~21 arcsec).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05659-b31b1b.svg)](https://arxiv.org/abs/2506.05659) | **The Observations of Magnetic Reconnection During the Interaction Process of Two Active Region Filaments**  |
|| Z. Wu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *16 pages, 7 figures*|
|**Abstract**|            We investigate the interaction between two filaments (F1 and F2) and their subsequent magnetic reconnection in active region (AR) NOAA 13296 and AR NOAA 13293 on May 9, 2023, utilizing high spatial and temporal resolution and multi-wavelength observational data from the Solar Dynamics Observatory, the New Vacuum Solar Telescope, and the Chinese H{\alpha} Solar Explorer. The movement of F1 from the southeast toward the northwest, driven by the motion of the positive magnetic polarity (P1), leads to a collision and reconnection with F2. This reconnection exchanges their footpoints, resulting in the formation of two new filaments (F3 and F4) consistent with "slingshot" type filament interaction. During the interaction, the current sheet moving due to the motion of F1 and the reconnection outflows moving along F3 and F4 were both observed. The current sheet is rarely observed in the slingshot type filament interaction, measuring approximately 2.17 Mm in length and 0.84 Mm in width. After the interaction, the F1 disappears whereas a portion of F2 remains, indicating that the interaction involves partial slingshot reconnection, due to the unequal magnetic flux between the filaments. The residual part of F2 will undergo another magnetic reconnection in the same interaction region with the magnetic loops connecting polarities N1 and P1. The material generated by the reconnection is continuously injected into F4, leading to its final morphology. The findings enhance our understanding of slingshot-type filament interactions, indicating that partial slingshot reconnections between filaments may be more common than full slingshot events.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05920-b31b1b.svg)](https://arxiv.org/abs/2506.05920) | **Multi-Instrument Search for Gamma-Ray Counterpart of X-ray Transients detected by EP/WXT**  |
|| Y.-Q. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *Accepted by ApJL*|
|**Abstract**|            As a soft X-ray imager with unprecedentedly large field of view, EP/WXT has detected many (fast) X-ray transients, whose nature is very intriguing. Whether there is gamma-ray counterpart for the X-ray transient provides important implications for its origin. Some of them have been reported to be associated with GRB, however, a systematic study on the gamma-ray emission of these X-ray transients is lacking. In this work, we implemented a comprehensive targeted search for gamma-ray counterparts to 63 X-ray transients reported by EP/WXT during its first year of operation, using the dedicated multiple-instrument search pipeline, ETJASMIN, with GECAM-B, GECAM-C, Fermi/GBM, and \textit{Insight}-HXMT data. We find that 14 out of 63 (22\%) EP/WXT X-ray transients have gamma-ray counterparts. For other transients, ETJASMIN pipeline provided upper limit of gamma-ray emission, which is more stringent than that given by individual instrument. Moreover, we investigated the properties of the X-ray transients and their gamma-ray counterparts, and explored the relation between the x-ray transient and gamma-ray counterpart.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06164-b31b1b.svg)](https://arxiv.org/abs/2506.06164) | **Venus Cloud Research: Progress and Perspectives**  |
|| L. Dai, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *76 pages, 14 figures*|
|**Abstract**|            Venus has regained attention on the international stage with the approval of three new missions by ESA and NASA. As the twin sister of Earth, Venus exhibits a distinct atmosphere, which casts a veil of mystery over the planetary evolution and is of great scientific significance. One of the most important components of Venus-the cloud-is believed to have significantly regulated its climate evolution and affect the environmental habitability. However, due to sparse in-situ measurements and the limitation of remote sensing, properties of these clouds remain largely unknown. Based on research conducted in past decades, this article reviews the observational structure of cloud properties, the progress of microphysical and simplified cloud model developments, and perspectives of future directions of this research field. Several possible solutions to the challenges associated with the coupling effect, ultraviolet absorption, and habitability are proposed and discussed in details, providing insights for future Venus' explorations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.05892-b31b1b.svg)](https://arxiv.org/abs/2506.05892) | **Disk Evolution Study Through Imaging of Nearby Young Stars (DESTINYS): Evidence of planet-disk interaction in the 2MASSJ16120668-3010270 system**  |
|| C. Ginski, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-06-09*|
|*Comments*| *14 pages, 10 figures, accepted for publication in A&A, abstract shortened to comply with arXiv format, associated ESO picture of the week release: this https URL*|
|**Abstract**|            The architectures of exoplanet systems are likely set during the initial planet-formation phase in the circumstellar disk. To understand this process, we have to study the earliest phases of planet formation. Complex sub-structures, believed to be driven by embedded planets, have been detected in a significant portion of disks observed at high angular resolution. We aim to extend the sample of such disks to low stellar masses and to connect the disk morphology to the expected proto-planet properties. We resolve the disk in the 2MASSJ16120668-3010270 system for the first time in scattered near-infrared light on scales of 10 au using VLT/SPHERE and reveal an exceptionally structured disk. We find an inner disk (inside 40 au) with two spiral arms, separated by a gap from an outer ring. By comparison with hydrodynamic models, we find that these structures are consistent with the presence of an embedded gas giant with a mass range between 0.1 and 5 MJup depending on the employed model. Our SPHERE observations find a tentative candidate point source within the disk gap, which may be consistent with this mass range if it indeed traces thermal emission by an embedded planet. This interpretation is somewhat strengthened by the proximity of this signal to compact mm continuum emission in the disk gap, which may trace circumplanetary material. It is, however, unclear if this tentative companion candidate could be responsible for the observed disk gap size, given its close proximity to the inner disk. The 2MASSJ16120668-3010270 system is one of only a few systems that shows this exceptional morphology of spiral arms located inside a scattered light gap and ring. We speculate that this may have to do with a higher disk viscosity compared with other systems such as PDS 70.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.05459/./figures/morph.png', 'tmp_2506.05459/./figures/z.png', 'tmp_2506.05459/./figures/euler.png', 'tmp_2506.05459/./figures/bar.png']
copying  tmp_2506.05459/./figures/morph.png to _build/html/
copying  tmp_2506.05459/./figures/z.png to _build/html/
copying  tmp_2506.05459/./figures/euler.png to _build/html/
copying  tmp_2506.05459/./figures/bar.png to _build/html/
exported in  _build/html/2506.05459.md
    + _build/html/tmp_2506.05459/./figures/morph.png
    + _build/html/tmp_2506.05459/./figures/z.png
    + _build/html/tmp_2506.05459/./figures/euler.png
    + _build/html/tmp_2506.05459/./figures/bar.png
found figures ['tmp_2506.05474/./figures/fig4_RP-AGN_Cases.jpg', 'tmp_2506.05474/./figures/fig5_RP-AGN_Median.jpg', 'tmp_2506.05474/./figures/fig9_Special_Case.jpg']
copying  tmp_2506.05474/./figures/fig4_RP-AGN_Cases.jpg to _build/html/
copying  tmp_2506.05474/./figures/fig5_RP-AGN_Median.jpg to _build/html/
copying  tmp_2506.05474/./figures/fig9_Special_Ca

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\rm \mu m}$
$\newcommand{\kms}{ \rm km s^{-1}}$
$\newcommand{\Msun}{\rm M_\odot}$
$\newcommand{\Ha}{{\rm H}\alpha\xspace}$
$\newcommand{\Nii}{[N {\sc ii}]\xspace}$
$\newcommand{\Hb}{{\rm H}\beta\xspace}$
$\newcommand{\Oiii}{[O {\sc iii}]\xspace}$
$\newcommand{\zspec}{z_{\rm spec}}$
$\newcommand{\zphot}{z_{\rm phot}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$</div>



<div id="title">

# RUBIES: A Spectroscopic Census of Little Red Dots: All Point Sources with V-Shaped Continua Have Broad Lines

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.05459-b31b1b.svg)](https://arxiv.org/abs/2506.05459)<mark>Appeared on: 2025-06-09</mark> -  _Main text: 17 pages, 10 figures. Appendix: 7 pages, 3 figures. Submitted to A&A_

</div>
<div id="authors">

R. E. Hviding, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** The physical nature of Little Red Dots (LRDs) -- a population of compact, red galaxies revealed by JWST -- remains unclear.    Photometric samples are constructed from varying selection criteria with limited spectroscopic follow-up available to test intrinsic spectral shapes and prevalence of broad emission lines.    We use the RUBIES survey, a large spectroscopic program with wide color-morphology coverage and homogeneous data quality, to systematically analyze the emission-line kinematics, spectral shapes, and morphologies of $\sim$ 1500 galaxies at $z > 3.1$ .    We identify broad Balmer lines via a novel fitting approach that simultaneously models NIRSpec/PRISM and G395M spectra, yielding 80 broad-line sources with 28 (35 \% ) at $z > 6$ .    A large subpopulation naturally emerges from the broad Balmer line sources, with 36 exhibiting `v-shaped' UV-to-optical continua and a dominant point source component in the rest-optical; we define these as spectroscopic LRDs, constituting the largest such sample to date.    Strikingly, the spectroscopic LRD population is largely recovered when either a broad line or rest-optical point source is required in combination with a v-shaped continuum, suggesting an inherent link between these three defining characteristics.    We compare the spectroscopic LRD sample to published photometric searches.    Although these selections have high accuracy, down to $\rm F444W<26.5$ , only 50-62 \% of the RUBIES LRDs were previously identified.    The remainder were missed due to a mixture of faint rest-UV photometry, comparatively blue rest-optical colors, or highly uncertain photometric redshifts.    Our findings highlight that well-selected spectroscopic campaigns are essential for robust LRD identification, while photometric criteria require refinement to capture the full population.

</div>

<div id="div_fig1">

<img src="tmp_2506.05459/./figures/morph.png" alt="Fig2" width="100%"/>

**Figure 2. -** Stellar locus and morphological classification in LW NIRCam filters. Black stars show the 95th percentile effective radius posterior ($r_{\rm eff, 95\%}$) vs. magnitude for reference stars, with the best-fit stellar locus (grey dashed). We classify galaxies as point sources (green hashed) if they fall below the $+4\sigma_\textrm{resid}$ offset of this relation ($r_\textrm{rsv}$; green line). Grey circles show all RUBIES sources with robust $\zspec>3.1$ while red hexagons highlight spectroscopic LRDs (Section \ref{sec:lrd}), plotted in the LW filter that traces the rest-5500Å filter depending on their redshift. (*fig:morph*)

</div>
<div id="div_fig2">

<img src="tmp_2506.05459/./figures/z.png" alt="Fig3" width="100%"/>

**Figure 3. -** Best-fit photometric versus spectroscopic redshift for RUBIES sources with robust $\zspec > 3.1$(grey histogram) and the LRD sample (red hexagons; triangle indicates out of plot). Photometric redshifts were measured using \texttt{eazy} ([Brammer and Coppi 2008]())  with two different template sets: the \texttt{blue\_sfhz\_13} set optimized for high-redshift galaxies (left) and the \texttt{agn\_blue\_sfhz\_13} set which additionally includes an AGN template constructed from the LRD of [Killi, Watson and Brammer (2024)]() and a torus model (right). Dashed lines show $|\Delta z|/(1+$\zspec$) = 0.1$. Although including this AGN template reduces the number of outliers and photometric redshift scatter for the LRD sample, a small bias toward higher photometric redshifts remains.  (*fig:photoz*)

</div>
<div id="div_fig3">

<img src="tmp_2506.05459/./figures/euler.png" alt="Fig8.1" width="50%"/><img src="tmp_2506.05459/./figures/bar.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Left: Euler diagram displaying the overlap between the three LRD characteristic criteria: a broad Balmer line, a dominant point source component at rest-optical wavelengths, and a spectroscopic v-shaped continuum. Right: Histogram showing the composition of overlaps in the Euler diagram where only two of the three features are measured.
    We note that objects with a rest-optical point source and a v-shaped continuum are 80\% likely to have a broad line.
    In fact, in the remaining 20\% we cannot reject a broad line, but our quality cuts restrict us from being fully confident.
    This suggests an underpinning physical link between the three LRD features. (*fig:euler*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.05459"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\swatch}[1]{\tikz[baseline=-0.6ex] \node[fill=#1,shape=rectangle,draw=black,thick,minimum width=5mm,rounded corners=2pt]() ;}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\includegraphics[scale=0.05]{figures/ORCID-iD_icon-128x128.png}}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Jellyfish galaxieswith the IllustrisTNG simulations --Supermassive black hole activity in dense environments with ram-pressure stripped satellites

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.05474-b31b1b.svg)](https://arxiv.org/abs/2506.05474)<mark>Appeared on: 2025-06-09</mark> -  _21 pages, 9 figures + 3 appendices with 3 figures. Submitted to MNRAS_

</div>
<div id="authors">

<mark>S. Kurinchi-Vendhan</mark>, et al. -- incl., <mark>E. Rohr</mark>, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** $\noindent$ Jellyfish galaxies are extreme examples of how galaxies can transform due to dense environmental effects. These satellite galaxies suffer from ram-pressure stripping, leading to the formation of their distinctive gaseous tails. Some recent observational studies find that jellyfish galaxies are more likely to host active galactic nuclei (AGN) compared to central galaxies of the same mass, suggesting a link between ram pressure and supermassive black hole (SMBH) accretion. We use the IllustrisTNG cosmological-magnetohydrodynamical simulations, namely TNG50 and TNG100, to explore the presence of AGN in jellyfish galaxies with $M_{\rm{stellar}}\simeq10^{9.5-10.8} \rm{M}_\odot$ at redshift $z=0$ from the Zooniverse "Cosmological Jellyfish" citizen-science project. Compared to central galaxies, jellyfish are more likely to host an AGN ( $L_{\rm AGN}\geq10^{44} \mathrm{erg s^{-1}}$ ) particularly at high stellar masses ( $M_{\rm stellar}\gtrsim10^{10} \mathrm{M_\odot}$ ). Jellyfish are also more likely to host an AGN than satellites of the same mass, largely because many satellite galaxies are gas-poor and therefore have lower SMBH accretion rates. Compared to non-jellyfish satellites with similar gas content, jellyfish typically undergo stronger ram pressure and have higher central gas densities along with lower central gas sound speeds, although these effects are smaller at lower stellar masses ( $M_{\rm stellar}\lesssim10^{10} \mathrm{M_\odot}$ ). Together with case studies of individual galaxies, our population analysis indicates that ram pressure can play a key role in fuelling AGN activity in a large fraction of jellyfish, where gas compression can lead to intense episodes of AGN feedback and star formation. Thus, it is essential to consider both environmental and secular processes for a more complete picture of satellite galaxy evolution.

</div>

<div id="div_fig1">

<img src="tmp_2506.05474/./figures/fig4_RP-AGN_Cases.jpg" alt="Fig10" width="100%"/>

**Figure 10. -** **_Ram-pressure feeding of SMBH activity,**_ shown through the evolution of ram pressure, central gas densities (innermost 1 ckpc), and AGN luminosities in three individual jellyfish galaxies in TNG50 at $z = 0$. We also note whether the galaxy is a jellyfish (yellow circle), inspected satellite (pink circle), satellite (blue circle), or central (purple circle) at each snapshot, as well as when the galaxy infalls into its halo (star; tracked by the host-centric distance in green). In all cases, most of the depicted quantities peak near in time to the first pericentric passage, where the host-centric distance is at a local minimum (dashed red line). **Case 1:** For \texttt{SubhaloID = 96788}, the peak in $P_{\rm ram}$ directly corresponds to a peak in $n_{\rm gas}$ and $L_{\rm AGN}$ at around the time of the first pericentric passage. **Case 2:** For \texttt{SubhaloID = 11}, the peaks in $n_{\rm gas}$ and $L_{\rm AGN}$ are slightly delayed compared to the peak in $P_{\rm ram}$, showing that ram pressure can indirectly effect SMBH activity after the pericentric passage. **Case 3:** For \texttt{SubhaloID = 282783}, the peak in $P_{\rm ram}$ corresponds to a peak in $n_{\rm gas}$ but not $L_{\rm AGN}$; here, it is possible for ram pressure to drive gas toward the centre of the jellyfish without affecting the SMBH. See Figure \ref{FIG:RP-AGN_Cumulative} for a cumulative evolution of all cases. (*FIG:RP-AGN*)

</div>
<div id="div_fig2">

<img src="tmp_2506.05474/./figures/fig5_RP-AGN_Median.jpg" alt="Fig1" width="100%"/>

**Figure 1. -** **_Ram-pressure feeding of SMBH activity,**_ shown through the evolution of ram pressure, central gas densities, and AGN luminosities in hundreds of jellyfish galaxies from TNG50 and TNG100 at $z = 0$. The cumulative i.e. "stacked" evolution represents a combination of three evolutionary paths by which ram pressure affects the AGN activity of the jellyfish galaxies: we categorize them as "Case 1" (Direct Effect), "Case 2" (Delayed Effect), and "Case 3" (No Effect) and quantify them in the above bar plot by visual inspection of the evolutionary tracks of all individual galaxies (as those of Figure \ref{FIG:RP-AGN}). We show the median-stacked evolution, with the galaxy-to-galaxy standard deviation around the median as the dashed outer lines. The standard error extends beyond the figure limits in the case of $n_{\rm gas}$ and $L_{\rm AGN}$, as these quantities are highly variable between galaxies. To stack the data, we shift the time axes by $\Delta \tau_{\rm infall}$ and scale by $f_{\rm peri. \: pass.}$ to align the evolutions at the average time of infall (dotted teal line) and first pericentric passage (dotted red line with 1-$\sigma$ shaded). The time correlation between ram pressure and AGN luminosity is somewhat weak. However, when we stack across only jellyfish galaxies of the first case (green circles), a strong and timely increase of AGN luminosity manifests with an increase of ram pressure and central gas density.
     (*FIG:RP-AGN_Cumulative*)

</div>
<div id="div_fig3">

<img src="tmp_2506.05474/./figures/fig9_Special_Case.jpg" alt="Fig12" width="100%"/>

**Figure 12. -** **_The special case of a satellite with ram-pressure stripped tails and an AGN-driven bubble.**_ The image shows the 2-D gas pressure projection of a satellite galaxy from TNG50 at $z = 0$, with \texttt{Subhalo ID = 184935, Snapshot 99} and a stellar mass of $M_{\rm stellar} = 10^{10.7} \: \rm{M}_\odot$. We show the trajectory of the satellite (arrows) as well as when it goes through a pericentric passage. It is orbiting a central galaxy with \texttt{Subfind ID = 184931} and a stellar mass of $M_{\rm stellar} = 10^{11.7} \: \rm{M}_\odot$, and both are residents of a larger halo structure with \texttt{Group Nr. = 6} and a host mass of $M_{\rm halo} = 10^{13.5} \: \rm{M}_\odot$. Not only does this satellite exhibit ram-pressure stripped tails, but also a bubble of high pressure gas at its leading edge. This looks similar to the eROSITA-like X-ray emitting bubbles of TNG50 Milky Way- and Andromeda-like galaxies driven by their SMBH kinetic feedback \citep[see][]{Pillepich_2021}. Interestingly, here the galaxy is a satellite and ram-pressure stripping, star formation, and gas bubbles driven by kinetic feedback from a low accretion SMBH all coexist at once. (*FIG:Special_Case*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.05474"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\masyr}{mas yr\ensuremath{^{-1}}\xspace}$
$\newcommand{\lsun}{\ensuremath{L_{\sun}}\xspace}$
$\newcommand{\micron}{\ensuremath{\mu}m\xspace}$
$\newcommand{\teff}{\ensuremath{T_\textrm{eff}}\xspace}$
$\newcommand{\re}{\ensuremath{R_\textrm{e}}\xspace}$
$\newcommand{\mh}{[M/H]\xspace}$
$\newcommand{\kms}{km s\ensuremath{^{-1}}\xspace}$
$\newcommand{\msun}{\ensuremath{M_{\sun}}\xspace}$
$\newcommand{\mbh}{\ensuremath{M_{\bullet}}\xspace}$
$\newcommand{\col}{\ensuremath{H-K_{S}}\xspace}$
$\newcommand{\vlos}{\ensuremath{V_\textrm{LOS}}\xspace}$
$\newcommand{\dlos}{\ensuremath{d_\textrm{LOS}}\xspace}$
$\newcommand{\slos}{\ensuremath{\sigma_{\textrm{LOS}}}\xspace}$
$\newcommand{\sgra}{Sgr~A^\star\xspace}$</div>



<div id="title">

# Dynamical mass distribution and velocity structure of the Galactic centre

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.06014-b31b1b.svg)](https://arxiv.org/abs/2506.06014)<mark>Appeared on: 2025-06-09</mark> -  _15 pages (+ 5 pages Appendix), 10 (+ 5) figures, accepted A&A_

</div>
<div id="authors">

A. Feldmeier-Krause, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** The inner $\sim$ 200 pc region of the Milky Way contains a nuclear stellar disc and a nuclear star cluster that are embedded in the larger Galactic bar. These stellar systems overlap spatially, which makes it challenging to separate stars that belong to the nuclear stellar systems, to deduce their internal dynamics, and to derive the central Galactic potential. Discrete stellar kinematics probe the mass distribution of a stellar system, and chemical tracers such as stellar metallicity can further separate multiple stellar populations that can have distinct kinematic properties.  We took advantage of the information provided by discrete stellar kinematics and the metallicity of stars in the Galactic centre using discrete chemo-dynamical modelling. We fitted axisymmetric Jeans models to discrete data of 4 600 stars. We fitted the stars as either one population plus a background component or as two populations plus a background that represents the bar. In the one-population case, we tested the robustness of the inferred gravitational potential against a varying mass of the supermassive black hole, including dark matter, or a radially varying mass-to-light ratio. We obtained robust results on the stellar dynamical fit with a single population and a background component. We obtained a supermassive black hole mass of (4.35 $\pm 0.24) \times 10^6$ $\msun$ , and we find that a dark matter component adds no more than a few percent to the total enclosed mass of the nuclear star cluster. The radial variation in the mass-to-light ratio is also negligible.  We derived the enclosed mass profile of the inner $\sim$ 60 pc of the Milky Way and found a lower mass than reported in the literature in the region of $\sim$ 5-30 pc.  In our two-population fit, we found a high- $\mh$ population with a mild tangentially anisotropic velocity distribution and stronger rotational support than for the low- $\mh$ population, which is radially anisotropic. The high- $\mh$ population is dominant and contributes more than 90 \% to the total stellar density. The properties of the high- $\mh$ population are consistent with in situ formation after gas inflow from the Galactic disc via the bar. The distinct kinematic properties of the low- $\mh$ population indicate a different origin.

</div>

<div id="div_fig1">

<img src="tmp_2506.06014/./emcee_corner21_bh2.png" alt="Fig11" width="100%"/>

**Figure 11. -** MCMC post burn-in  distributions for the one-population models with $\mbh$ as free parameter  (Sect. $\re$f{sec:fitbh}). The scatter plots show the projected two-dimensional distributions, the red lines the respective 50th percentile, and the dashed grey lines the 16th and 84th percentiles. Their values are also written on top of each column, displaying the projected one-dimensional distributions. From top to bottom and left to right, the panels show the inner anisotropy $\beta_0$, outer anisotropy $\beta_\infty$, the anisotropy transition radius $\log(R_\beta)$, inner rotation parameter $\kappa_0$, outer rotation parameter $\kappa_\infty$, the rotation transition radius $\log(R_\kappa)$, the mass-to-light ratio $\Upsilon$, fraction of background stars $\epsilon$ in percent, and the black hole mass $\mbh$ in $10^6$\msun.  (*fig:cornerbh*)

</div>
<div id="div_fig2">

<img src="tmp_2506.06014/./betakappaml_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Radial profile of the velocity anisotropy $\beta_z$(top), the rotation $\kappa$(middle), and
  the mass-to-number density conversion $\Upsilon$(bottom) as derived in Sect. $\re$f{sec:fitmlr} using a three-parameter function. The red lines denote the median of the posterior distribution, and the grey lines are 50 randomly drawn realisations from the posterior distribution. The vertical solid lines denote 1 $\re$=5 pc, the dotted lines show the outer limit of the kinematic data, and the dot-dashed lines show the median of $R_\beta$, $R_\kappa$, and $R_\Upsilon$, respectively.
 (*fig:betakappaml*)

</div>
<div id="div_fig3">

<img src="tmp_2506.06014/./model12_mhsigvxyz_2.png" alt="Fig10" width="100%"/>

**Figure 10. -** Spatial distribution of stars, colour-coded from top to bottom by the velocity dispersion along $l$, along $b$, along the line of sight, $\vlos$, $\vlos$/$\slos$, for the 50th percentile two-population model, with high-$\mh$ stars (left), and with low-$\mh$ stars (right).
   (*fig:modelmh*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.06014"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Faint absorption of the ground state hyperfine-splitting transitions of hydroxyl at 18 cm in the Galactic Disk$\thanks{This work is based on observations made with the Karl G. Jansky Very Large Array under the project VLA 17A-055.  An earlier version of parts of this work appeared in the PhD thesis of M. R. Rugel.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.06149-b31b1b.svg)](https://arxiv.org/abs/2506.06149)<mark>Appeared on: 2025-06-09</mark> -  _13 pages, 7 figures (appendices excluded). Accepted for publication in A&A_

</div>
<div id="authors">

M. R. Rugel, et al. -- incl., <mark>H. Beuther</mark>, <mark>Y. Wang</mark>

</div>
<div id="abstract">

**Abstract:** The interstellar hydride hydroxyl (OH) is a potential tracer of CO-dark molecular gas. We present new high-sensitivity absorption line observations of the four ground state hyperfine-splitting transitions of OH at 18-cm wavelength towards four Galactic and extra-galactic continuum sources as follow-up to the THOR survey. We compare these to deep observations of the [ $\ion{C}{ii}$ ] 158 $\mu$ m line at 1.9 THz obtained with the upGREAT instrument on SOFIA, observations of the neutral atomic hydrogen ( $\ion{H}{i}$ ) 21 cm line with the VLA, and CO ( $J$ = 2--1) lines obtained with the APEX PI230 receiver at the APEX 12 m sub-mm telescope. We trace OH over a large range of molecular hydrogen column densities of 7.9 $\times10^{19}$ cm $^{-2}$ to 4.7 $\times10^{22}$ cm $^{-2}$ , and derive OH abundances with respect to molecular and total hydrogen column densities of X $_{\rm OH, H_2} =N_{\rm OH}/N_{\rm H_2} = 1.2^{+0.3}_{-0.2}\times10^{-7}$ and $X_{\rm OH, H} = N_{\rm OH}/N_{\rm H}=4.8^{+0.9}_{-0.8}\times10^{-8}$ , respectively. Increased sensitivity and spectral resolution allowed us to detect weak and narrow features with the lowest column density detected at $N_{\rm OH}= 3.7\times10^{13}$ cm $^{-2}$ . The increase in sensitivity is a factor of 5 in direct comparison at the resolution the OH observations in the THOR survey (1.5 $\kms$ ). We identify only one OH absorption component out of 23 without CO counterpart, yet several with intermediate molecular gas fractions ( ${\rm f_{mol}\leq0.8}$ ). A potential association of [ $\ion{C}{ii}$ ] 158 $\mu$ m emission with an OH absorption component is seen toward one sightline. Our results confirm that OH absorption traces molecular gas across diffuse and dense environments of the interstellar medium. At the sensitivity limits of the present observations our detection of only one CO-dark molecular gas feature appears in agreement with previous studies. We conclude that if OH absorption was to be used as a CO-dark molecular gas tracer, deeper observations or stronger background targets are necessary to unveil its full potential as a CO-dark molecular gas tracer, and yet it will never be an exclusive tracer of CO-dark molecular gas. For OH hyperfine-splitting transitions in the vicinity of photodissociation regions in W43-South, we detect a spectral and spatial offset between the peak of the inversion of the OH 1612 MHz line and the absorption of the OH 1720 MHz line on the one hand, and the absorption of the OH main lines on the other hand, which provides additional constraints on the interpretation of the OH 18 cm line signatures typical of HII regions.

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="50%"/><img src="" alt="Fig5.2" width="50%"/>

**Figure 5. -** _ Top:_ THOR 1.4 GHz continuum emission in the Galactic coordinate frame  ([Wang, Beuther and Rugel 2020]()) . The sources discussed here are located at the center of the yellow circles. _ Bottom:_ VLA 1.4 GHz continuum towards the four positions analyzed in this work _ (blue crosses)_ in equatorial coordinates. The images are smoothed to an angular resolution of 18$\arcsec$. (*fig:sample_selection*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.06149"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

514  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
